# Connect with Google Drive service
https://developers.google.com/drive/api/guides/search-files

Need to share google folder with the client email: google-drive-api@###gserviceaccount.com

In [ ]:
# import neccessary packages
import os
import io
import pandas as pd
from dotenv import load_dotenv
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name("../../../gsheet-service-credentials.json", SCOPES)
service = build('drive', 'v3', credentials=creds)

In [ ]:
resource = service.files()
result = resource.list(pageSize=5, fields="files(id, name)",q="mimeType = 'application/vnd.google-apps.folder'").execute()

In [ ]:
PUD_folder_id = [x['id'] for x in result['files'] if x['name'] == 'Phone Usage Data'][0]

In [ ]:
file_list = resource.list( fields="files(id, name)",q="name contains 'V4_Usage'").execute()

In [ ]:
from datetime import datetime
def build_usage_df(df, file_content):
    start = False
    access_count = 0
    num_apps = 0
    for i in file_content:
        if start == True and i == '""':
            start = False
            break
        elif start == True:
            items = i.split(",")
            num_apps+=1
            access_count+= int(items[-1].replace('"', ''))
        elif start == False and "Access count" in i:
            start = True
    usage_index = 1 + [i for i in range(1,len(file_content)) if "Usage time" in file_content[i]][0]
    date = datetime.strptime(file_content[usage_index].split(",")[0].replace('"', ''), '%m/%d/%y')
    usage_list = file_content[usage_index].split(",")[1].split(":")
    usage = int(usage_list[0].replace('"', ''))*60*60+int(usage_list[1].replace('"', ''))*60+int(usage_list[2].replace('"', ''))
    df = pd.concat([df, pd.DataFrame.from_records([{'Date' : date, 'Usage Time' : usage, 'Access Count' : access_count,'Number of Apps':num_apps}])],ignore_index=True)
    return df

In [ ]:
df_usage = pd.DataFrame(columns = ['Date', 'Usage Time', 'Access Count', 'Number of Apps'])
for file in file_list['files']:
    request  = service.files().get_media(fileId=file['id'])
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    file_content = str( file.getvalue(), 'UTF-8')
    file_content = file_content.split("\n")
    df_usage = build_usage_df(df_usage,file_content)

In [ ]:
df_usage